<a href="https://colab.research.google.com/github/Noxara84/mineria-de-datos/blob/master/PE_Jacobo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Conversión de variables descriptivas en numéricas usando distribución de probabilidad acumulada**




Vamos a tomar una base de datos predeterminada para ejemplificar el método. Para ello necesitamos la librería seaborn. Además de ocupar las librerías pandas y numpy para hacer los cálculos y transformaciones dentro del método.

In [0]:
import pandas as pd
import seaborn as sb
import numpy as np 
from random import gauss

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Cargamos nuestra base de datos a una variable llamada df2. La base de datos que elegimos es titanic y contiene datos sobre el accidente de la embarcación con el mismo nombre. 

Visualizamos los datos contenidos en la base de datos. Principalmente para poder identificar las variables descriptivas y las numéricas. 

In [0]:
df2= sb.load_dataset('titanic')
df2.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Se puede observar que existen distintas columnas con valores descriptivos (*sex, embarked, class, who, adult_male, deck, embark_town, alive y alone*).

In [0]:
df2.embark_town.value_counts()

Southampton    644
Cherbourg      168
Queenstown      77
Name: embark_town, dtype: int64

Realizamos una evaluación del embark_town. Notamos que existen tres pueblos en toda la base de datos. De los cuáles 644 son de Shothampton, 168 de Cherbourg y 77 de Queenstown. 

In [0]:
#Jaja a mano saqué esto solo para visualizarlo
#N=889
#S=0.7244
#C=0.1889
#Q=0.0866

Lo siguiente es guardar el conteo de la columna embark_town en una variable llamada a. 

In [0]:
a = df2.embark_town.value_counts()

In [0]:
type(a)

pandas.core.series.Series

Imprimiendo de vuelta los valores contenidos en a, están las frecuencias de los pueblos como lo vimos anteriormente. 

In [0]:
for x in a:
    print(x)

644
168
77


Lo siguiente es crear una variable N en donde se concentren todos los casos. Esto nos ayudará más adelante a calcular las probabilidades de cada uno de los pueblos dentro de la columna. 

In [0]:
N = df2['embark_town'].count()

Una vez teniendo todos los ingredientes, lo siguiente es realizar la transformación de los datos. Para esto, se crearán dos diccionarios, uno será llamado como tal diccionario y el otro se llamará frecuencias. El diccionario "diccionario" le dará valores numéricos a nuestras ciudades, con la finalidad de ser más manejable el proceso de conversión. En lugar de tener ahora los pueblos, tendremos valores (0,1,2) respectivamente. 
Mientras que el diccionario "frecuencias" se encarga de de tomar su valor numérico y dividirlo entre el total de datos. 

Finalmente la salida son las frecuencias, en donde vemos dos valores perdidos. Estos parecieran ser valores que no tienen un pueblo asignado pero que siguen siendo datos dentro de la tabla.  
El último bloque muestra el nombre del pueblo, seguido de la probabilidad de ocurrencia dentro de la base de datos. 

In [0]:
diccionario = {}
frecuencias = {}

for x in df2['embark_town']:
  diccionario[x] = 0; 
  
for x in df2['embark_town']:
  diccionario[x] = diccionario[x] + 1
  
for x in diccionario:
  print(diccionario[x])
  frecuencias[x] =  diccionario[x]/N

for x in diccionario:
  print(str(x) + ' ' + str(frecuencias[x]))


644
168
77
2
Southampton 0.7244094488188977
Cherbourg 0.1889763779527559
Queenstown 0.08661417322834646
nan 0.0022497187851518562


El último paso, es crear los intervalos. Para ello utilizamos un ciclo for, en el cuál toma la frecuencia en cero y le suma la frecuencia del diccionario "frecuencias". La salida serán entonces a los intervalos de probabilidad acumulado en los que se encuentran los pueblos. 

In [0]:
intervalos = {}
inicio = 0;
for x in diccionario:
  intervalos[x] = [inicio, inicio + frecuencias[x]]
  inicio = inicio + frecuencias[x]

for x in diccionario:
  print(intervalos[x])

[0, 0.7244094488188977]
[0.7244094488188977, 0.9133858267716536]
[0.9133858267716536, 1.0]
[1.0, 1.0022497187851518]


In [0]:
#dist_est = {}
#media = {}
#for x in diccionario:
#  dist_est[x] = ((intervalos[x][0] + intervalos[x][1])/6)**2
#  media[x] = (intervalos[x][0] + intervalos[x][1])/2
#
#for x in diccionario:
#  print(str(dist_est[x]) + ' ' + str(media[x]))

0.014576918042724978 0.36220472440944884
0.07451037124296474 0.8188976377952757
0.10169570339140681 0.9566929133858268
0.11136122045486722 1.001124859392576


Reevaluación de la tabla


In [0]:
df_copy = df2
for x in frecuencias:
  df_copy.replace(to_replace =x, value =frecuencias[x])

df_copy["embark_town"].replace(frecuencias, inplace=True)
df_copy 

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,0.724409,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,0.188976,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,0.724409,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,0.724409,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,0.724409,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,0.724409,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,0.724409,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,0.724409,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,0.188976,yes,True


In [0]:
#df_copy = df2
#for x in df_copy['embark_town']:
#  x = gauss(media[x], dist_est[x])
#df_copy

In [0]:
df2

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [0]:
list(df2['embark_town'].unique())

[1.0, 2.0, 3.0, nan]

In [0]:
keys= list(df2['embark_town'].unique())
vals= range(1,4)
act2= dict(zip(keys,vals))
act2

{1.0: 1, 2.0: 2, 3.0: 3}